In [ ]:
!git clone https://huggingface.co/Makima57/deepseek-math-Numina

In [ ]:
! pip install ctranslate2

In [2]:
import pandas as pd
import ctranslate2
from transformers import AutoTokenizer
import time

# Define the prompt to be used by the model
model_prompt = "Solve the following mathematical problem with short steps "

# Load the tokenizer from the specified path
print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained("AI-MO/NuminaMath-7B-TIR")

# Load the CTranslate2 model for autoregressive text generation
print("Loading CTranslate2 model...")
model_path = "/kaggle/working/deepseek-math-Numina"
generator = ctranslate2.Generator(model_path, device="cuda",compute_type="int8")  # Use "cuda" if using GPU

# Load the dataset
print("Loading dataset...")
###CHANGE CSV FILE
dataset_path = "/kaggle/input/deepseek-math-1/combined_problems_solutions_part_8.csv"
df = pd.read_csv(dataset_path)

# Prepare lists to store results
questions = []
predicted_answers = []
actual_answers = []

# Start timing the process
start_time = time.time()

# Iterate over the entire dataset rows
print("Starting prediction process...")
for index, row in df.iterrows():
    question = row['problem']
    actual_answer = row['solution']  # Assuming 'solution' is the actual answer column
    
    # Add the model prompt to the question
    input_text = model_prompt + question
    
    # Tokenize the input text
    input_tokens = tokenizer.tokenize(input_text)
    
    # Generate prediction using the model
    results = generator.generate_batch([input_tokens],max_length=4096)
    
    # Detokenize the output tokens to get the predicted answer
    output_tokens = results[0].sequences[0]
    predicted_answer = tokenizer.convert_tokens_to_string(output_tokens)
    
    # Store the results
    questions.append(question)
    predicted_answers.append(predicted_answer)
    actual_answers.append(actual_answer)
    print(actual_answer)
    break
    

# # Create a new DataFrame with the results
# result_df = pd.DataFrame({
#     'question': questions,
#     'predicted_answer': predicted_answers,
#     'actual_answer': actual_answers
# })

# # Save the results to a new CSV file
# output_path = "/kaggle/working/predicted_answers_full_part8.csv"
# result_df.to_csv(output_path, index=False)

# # Print final completion message
# total_time = time.time() - start_time
# print(f"Predictions saved to: {output_path}")
# print(f"Total time taken: {total_time:.2f} seconds")


Loading tokenizer...


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading CTranslate2 model...
Loading dataset...
Starting prediction process...
Let $K$ be the intersection of $BC$ and $AE$ . Since the radii of the two circles are 1:4, so we have $AD:AE=1:4$ , and the distance from $B$ to line $l$ and the distance from $C$ to line $l$ are in a ratio of 4:1, so $BK:CK=4:1$ . We can easily calculate the length of $BC$ to be 4, so $CK=\frac{4}{3}$ . Let $J$ be the foot of perpendicular line from $A$ to $BC$ , we can know that $BJ:CJ=1:4$ , so $BJ = 0.8$ $CJ=3.2$ $AJ=1.6$ , and $AK=\sqrt{1.6^2+\left(3.2+\frac{4}{3}\right)^2}=\frac{4}{3}\sqrt{13}$ . Since $CK^2 = EK\cdot AK$ , so $EK=\frac{4}{39}\sqrt{13}$ , and $AE = \frac{4}{3}\sqrt{13} - \frac{4}{39}\sqrt{13}= \frac{16}{13}\sqrt{13}$ $\sin\angle AKB=\frac{AJ}{AK} = \frac{1.6}{\frac{4}{3}\sqrt{13}}=\frac{1.2}{\sqrt{13}}$ , so the distance from $C$ to line $l$ is $d=CK\cdot \sin\angle AKB = \frac{4}{3}\cdot \frac{1.2}{\sqrt{13}}=\frac{1.6}{\sqrt{13}}$ . so the area is \[[ACE] = \frac{1}{2}\cdot AE\cdot d

In [5]:
# Iterate over the entire dataset rows
print("Starting prediction process...")

model_prompt = "Solve the following mathematical problem with short steps and python code. "

question = "What is the area of a circle if its radius is 4 cm?"

# Combine the prompt and question
input_text = model_prompt + question

# Tokenize the input text (clear any previous tokens)
input_tokens = tokenizer.tokenize(input_text)

# Generate prediction using the model with a fresh input
results = generator.generate_batch([input_tokens], max_length=4096)

# Extract the generated tokens
output_tokens = results[0].sequences[0]

# Detokenize the output tokens to get the predicted answer
predicted_answer = tokenizer.convert_tokens_to_string(output_tokens)

# Print the predicted answer
print(predicted_answer)



Starting prediction process...
Solve the following mathematical problem with short steps and python code. What is the area of a circle if its radius is 4 cm? Use π = 3.14.
### Solution: To find the area of a circle given its radius, we use the formula:

\[ \text{Area} = \pi r^2 \]

Given:
- Radius \( r = 4 \) cm
- \(\pi = 3.14\)

Let's calculate the area using this formula.

Here's the step-by-step solution:

1. Square the radius.
2. Multiply the squared radius by \(\pi\).

We'll implement this in Python to ensure accuracy.
```python
# Given values
radius = 4  # in cm
pi = 3.14

# Calculate the area of the circle
area = pi * (radius ** 2)
print(area)
```
```output
50.24
```
The area of the circle with a radius of 4 cm is \(\boxed{50.24}\) square centimeters.



## Code to execute python code 


In [8]:
import os
import re
import signal
import subprocess
import tempfile
from collections import Counter
from contextlib import contextmanager
from dataclasses import dataclass

In [9]:
class PythonREPL:
    def __init__(self, timeout=5):
        self.timeout = timeout

    @contextmanager
    def time_limit(self, seconds):
        def signal_handler(*_):
            raise TimeoutError(f"Timed out after {seconds} seconds.")

        signal.signal(signal.SIGALRM, signal_handler)
        signal.alarm(seconds)
        try:
            yield
        finally:
            signal.alarm(0)

    def __call__(self, query):
        query = "import math\nimport numpy as np\nimport sympy as sp\n" + query
        query = query.strip().split("\n")
        if "print(" not in query[-1]:
            if "#" in query[-1]:
                query[-1] = query[-1].split("#")[0]
            query[-1] = "print(" + query[-1] + ")"
        query = "\n".join(query)
        with tempfile.TemporaryDirectory() as temp_dir:
            temp_file_path = os.path.join(temp_dir, "tmp.py")
            with open(temp_file_path, "w", encoding="utf-8") as f:
                f.write(query)
            with self.time_limit(self.timeout):
                result = subprocess.run(
                    ["python3", temp_file_path],
                    capture_output=True,
                    check=False,
                    text=True,
                    timeout=self.timeout,
                )
                if result.returncode == 0:
                    output = result.stdout
                    return True, output.strip()
                error_msg = result.stderr.strip()
                msgs = error_msg.split("\n")
                new_msgs = []
                want_next = False
                for m in msgs:
                    if "Traceback" in m:
                        new_msgs.append(m)
                    elif m == msgs[-1]:
                        new_msgs.append(m)
                    elif temp_file_path in m:
                        st = m.index('"/') + 1 if '"/' in m else 0
                        ed = m.index(temp_file_path) + 1 if temp_file_path in m else None
                        clr = m[st:ed] if not ed else m[st:]
                        m = m.replace(clr, "")
                        new_msgs.append(m)
                        want_next = True
                    elif want_next:
                        new_msgs.append(m)
                        want_next = False
                error_msg = "\n".join(new_msgs)
                return False, error_msg.strip()
            

def execute_completion(executor, completion, return_status, last_code_block):
    executions = re.findall(r"```python(.*?)```", completion, re.DOTALL)
    if len(executions) == 0:
        return completion, False if return_status else completion
    if last_code_block:
        executions = [executions[-1]]
    outputs = []
    successes = []
    for code in executions:
        success = False
        for lib in ("subprocess", "venv"):
            if lib in code:
                output = f"{lib} is not allowed"
                outputs.append(output)
                successes.append(success)
                continue
        try:
            success, output = executor(code)
        except TimeoutError as e:
            print("Code timed out")
            output = e
        if not success and not return_status:
            output = ""
        outputs.append(output)
        successes.append(success)
    output = str(outputs[-1]).strip()
    success = successes[-1]
    if return_status:
        return output, success
    return output


def postprocess_completion(text, return_status, last_code_block):
    executor = PythonREPL()
    result = execute_completion(executor, text, return_status=return_status, last_code_block=last_code_block)
    del executor
    return result

In [11]:
answer=postprocess_completion(predicted_answer,True,True)
print(answer)

('50.24', True)


## code to extract boxed answer


In [ ]:
def extract_boxed_answer(text):
    def last_boxed_only_string(text):
        idx = text.rfind("\\boxed")
        if idx < 0:
            idx = text.rfind("\\fbox")
            if idx < 0:
                return None
        i = idx
        right_brace_idx = None
        num_left_braces_open = 0
        while i < len(text):
            if text[i] == "{":
                num_left_braces_open += 1
            if text[i] == "}":
                num_left_braces_open -= 1
                if num_left_braces_open == 0:
                    right_brace_idx = i
                    break
            i += 1
        if right_brace_idx is None:
            return None
        return text[idx : right_brace_idx + 1]

    def remove_boxed(boxed):
        left = "\\boxed{"
        try:
            assert boxed[: len(left)] == left
            assert boxed[-1] == "}"
            length = len(left)
            return boxed[length:-1]
        except Exception:
            return None

    boxed = last_boxed_only_string(text)
    if boxed is None:
        return None
    answer = remove_boxed(boxed)
    return answer


def normalize_answer(answer):
    match = re.search(r"(.*?)Problem:", answer, flags=re.S)
    if match:
        answer = match.group(1)
    subs = [("an ", ""), ("a ", ""), (".$", "$"), ("\\$", ""), (r"\ ", ""), (" ", ""), ("mbox", "text"), (",\\text{and}", ","), ("\\text{and}", ","), ("\\text{m}", "\\text{}"), ("\\le", "<")]
    remove = ["square", "ways", "integers", "dollars", "mph", "inches", "ft", "hours", "km", "units", "\\ldots", "sue", "points", "feet", "minutes", "digits", "cents", "degrees", "cm", "gm", "pounds", "meters", "meals", "edges", "students", "childrentickets", "multiples", "\\text{s}", "\\text{.}", "\\text{\ns}", "\\text{}^2", "\\text{}^3", "\\text{\n}", "\\text{}", r"\mathrm{th}", r"^\circ", r"^{\circ}", r"\;", r",\!", "{,}", '"', "\\dots", "\n", "\r", "\f", "\%"]
    sub_patterns = [r"(\\text\{)(.*?)(\})", r"(\\textbf\{)(.*?)(\})", r"(\\overline\{)(.*?)(\})", r"(\\boxed\{)(.*)(\})"]
    split_patterns = [r"finalansweris(.*)", r"answer?is:?(.*)", r"oxed\{(.*?)\}", r"\$(.*?)\$"]
    for before, after in subs:
        answer = answer.replace(before, after)
    for expr in remove:
        answer = answer.replace(expr, "")
    for pattern in sub_patterns:
        answer = re.sub(pattern, "\\2", answer)
    for pattern in split_patterns:
        if len(re.findall(pattern, answer)) > 0:
            answer = re.findall(pattern, answer)[-1]
    answer = answer.strip()
    if "rac" in answer and "\\frac" not in answer:
        answer = answer.replace("rac", "\\frac")
    answer = re.sub(r"(frac)([^{])(.)", "frac{\\2}{\\3}", answer)
    answer = re.sub(r"(sqrt)([^{])", "sqrt{\\2}", answer)
    answer = answer.replace("$", "")
    if answer.replace(",", "").isdigit():
        answer = answer.replace(",", "")
    return answer

#mathematics evaluation

In [12]:
!git clone https://github.com/MARIO-Math-Reasoning/MARIO_EVAL.git
!cd MARIO_EVAL
!cd latex2sympy && pip install . && cd ..
!pip install -e .

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Cloning into 'MARIO_EVAL'...
remote: Enumerating objects: 238, done.
remote: Counting objects: 100% (238/238), done.
remote: Compressing objects: 100% (165/165), done.
remote: Total 238 (delta 97), reused 210 (delta 69), pack-reused 0 (from 0)
Receiving objects: 100% (238/238), 3.57 MiB | 32.11 MiB/s, done.
Resolving deltas: 100% (97/97), done.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/bin/bash: line 0: cd: latex2sympy: No such file or directory


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Obtaining file:///kaggle/working
ERROR: file:///kaggle/working does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.


In [15]:
%cd /kaggle/working/MARIO_EVAL

/kaggle/working/MARIO_EVAL


In [22]:
from math_evaluation import is_equiv


In [18]:
!pip install sympy==1.12
!pip install antlr4-python3-runtime==4.11.1

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 70.4 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: sympy
    Found existing installation: sympy 1.13.0
    Uninstalling sympy-1.13.0:
      Successfully uninstalled sympy-1.13.0


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached antlr4_python3_runtime-4.11.1-py3-none-any.whl.metadata (291 bytes)
Using cached antlr4_python3_runtime-4.11.1-py3-none-any.whl (144 kB)


In [21]:
!pip install timeout_decorator

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Preparing metadata (setup.py) ... done
  Created wheel for timeout_decorator: filename=timeout_decorator-0.5.0-py3-none-any.whl size=5004 sha256=722bebbb9149fb43253296081a7e945c722cb24be117f9c8223796a5c31dc79f
  Stored in directory: /root/.cache/pip/wheels/68/2f/bc/76f1192d474666d41ae6f09813fccbd00fe3f07e8261c4cff5
Successfully built timeout_decorator


## check if equivalent 

In [42]:
chatgpt="""
# Define the symbol for radius
r = sp.Symbol('r')

# Define the formula for the area of a circle (A = π * r^2)
area = sp.pi * r**2

# Substitute the value of radius r = 4 cm
result = area.subs(r, 4)

# Display the result
print(f"The area of the circle is {result} cm².")"

"""
chatgpt="""r = 4
pie = 3.14
area = pie * (r ** 2)
print(area)"""


In [23]:
is_equiv("1\\frac12", "1.5")



True

In [24]:
is_equiv("\\begin{pmatrix} 1 & \\frac12 \\\\ 1/3 & \\sqrt4 \\end{pmatrix}", 
         "[[1.0, 1/2],[0.3333, 2.0]]")

True

In [43]:
import re

code_pattern = r'```(?:python)?(.*?)```'

# Find all matches for the code blocks
matches = re.findall(code_pattern,predicted_answer, re.DOTALL)
print(matches)

# Print the extracted code
for match in matches:
    code=filter_code(match)
    equi=is_equiv(chatgpt,code)
    print(equi)
    print(code)


['\n# Given values\nradius = 4  # in cm\npi = 3.14\n\n# Calculate the area of the circle\narea = pi * (radius ** 2)\nprint(area)\n', 'output\n50.24\n']
False
radius = 4
pi = 3.14
area = pi * (radius ** 2)
print(area)
False
output
50.24


In [32]:
query= match
query = query.strip().split("\n")
if "print(" not in query[-1]:
    if "#" in query[-1]:
        query[-1] = query[-1].split("#")[0]
        query[-1] = "print(" + query[-1] + ")"
query = "\n".join(query)
print(query)

output
50.24


In [37]:
import re

# Input string with newlines and comments

def filter_code(input_str):

# Split the input by lines
    lines = input_str.split("\n")

    # Filter the lines: remove lines starting with # and strip inline comments
    filtered_lines = []
    for line in lines:
        # Remove inline comments and strip whitespace
        line = re.sub(r'#.*', '', line).strip()
        if line:  # If the line is not empty after stripping
            filtered_lines.append(line)

    # Join the filtered lines back into a single string, separated by newline characters
    cleaned_code = "\n".join(filtered_lines)
    return cleaned_code


## from ast tree

In [47]:
import ast

# Define the two code snippets
code1 = """
r = sp.Symbol('r')
area = sp.pi * r**2
result = area.subs(r, 4)
print(f"The area of the circle is {result} cm².")
"""

code2 = """
r = 4
pie = 3.14
area = pie * (r ** 2)
print(area)
"""

# Parse the code snippets into AST
tree1 = ast.parse(code1)
tree2 = ast.parse(code2)

# Function to check if two AST trees are structurally similar
def compare_ast(tree1, tree2):
    return ast.dump(tree1) == ast.dump(tree2)

# Compare the two code snippets
are_equal = compare_ast(tree1, tree2)
print(f"The code snippets are structurally similar: {are_equal}")


The code snippets are structurally similar: False


In [49]:
!pip install astor

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [54]:
import ast
import astor

# Define the two code snippets
code1 = """
r = sp.Symbol('r')
area = sp.pi * r**2
result = area.subs(r, 4)
print(f"The area of the circle is {result} cm².")
"""

code2 = """
r = 4
pie = 3.14
area = pie * (r ** 2)
print(area)
"""

def normalize_code(code):
    """Replace variable names with generic placeholders."""
    tree = ast.parse(code)
    class Normalizer(ast.NodeTransformer):
        def __init__(self):
            self.counter = 0
        
        def visit_Name(self, node):
            # Replace variable names with generic placeholders
            new_name = f"var{self.counter}"
            self.counter += 1
            return ast.copy_location(ast.Name(id=new_name, ctx=node.ctx), node)
    
    normalizer = Normalizer()
    normalized_tree = normalizer.visit(tree)
    return astor.to_source(normalized_tree)

# Normalize both code snippets
normalized_code1 = normalize_code(code1)
normalized_code2 = normalize_code(code2)

# Parse the normalized code snippets into AST
tree1 = ast.parse(normalized_code1)
tree2 = ast.parse(normalized_code2)

# Function to check if two AST trees are structurally similar
def compare_ast(tree1, tree2):
    return ast.dump(tree1) == ast.dump(tree2)

# Compare the two normalized code snippets
are_equal = compare_ast(tree1, tree2)
print(f"The code snippets are structurally similar: {are_equal}")

# Optionally print normalized code for debugging
print(f"Normalized code1:\n{normalized_code1}")
print(f"Normalized code2:\n{normalized_code2}")


The code snippets are structurally similar: False
Normalized code1:
var0 = var1.Symbol('r')
var2 = var3.pi * var4 ** 2
var5 = var6.subs(var7, 4)
var8(f'The area of the circle is {var9} cm².')

Normalized code2:
var0 = 4
var1 = 3.14
var2 = var3 * var4 ** 2
var5(var6)

